In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

import setting_for_ccle as setting
# import setting_for_tcga as setting

path_dict = path(setting)

In [ ]:
w = pd.read_table(
    path_dict['w_file_path'],
    index_col=0,
)

h = pd.read_table(
    path_dict['h_file_path'],
    index_col=0,
)

In [ ]:
setting.ELEMENT_ENTROPY_QUANTILE = 1.0

if setting.ELEMENT_ENTROPY_QUANTILE < 1:

    feature_entropy = w.apply(
        ccal.compute_entropy,
        axis=1,
    )

    w = w.loc[feature_entropy.index[feature_entropy < feature_entropy.quantile(q=setting.ELEMENT_ENTROPY_QUANTILE)]]

    print(w.shape)

    sample_entropy = h.apply(ccal.compute_entropy)

    h = h[sample_entropy.index[sample_entropy < sample_entropy.quantile(q=setting.ELEMENT_ENTROPY_QUANTILE)]]

    print(h.shape)

In [ ]:
gps_map = ccal.GPSMap(
    wt=w.T,
    h=h,
    wt_pull_power=setting.GPS_MAP_WT_PULL_POWER,
    h_pull_power=setting.GPS_MAP_H_PULL_POWER,
    plot=False,
)

In [ ]:
for wt_or_h, hcc_k, bandwidth_factor, element_marker_size in (
#     (
#         'wt',
#         setting.WT_HCC_K,
#         setting.GPS_MAP_WT_BANDWIDTH_FACTOR,
#         setting.GPS_MAP_WT_ELEMENT_MARKER_SIZE,
#     ),
    (
        'h',
        setting.H_HCC_K,
        setting.GPS_MAP_H_BANDWIDTH_FACTOR,
        setting.GPS_MAP_H_ELEMENT_MARKER_SIZE,
    ),
):

    gps_map.plot_gps_map(
        wt_or_h,
        title=setting.NAME,
        element_marker_size=element_marker_size,
        html_file_path='{}/map.html'.format(path_dict['{}_map_directory_path'.format(wt_or_h)]),
        plotly_file_path=path_dict['{}_map_plotly_file_path'.format(wt_or_h)],
    )

    hcc__k_x_column_file_path = path_dict['{}_hcc__k_x_column_file_path'.format(wt_or_h)]

    if os.path.isfile(hcc__k_x_column_file_path):

        hcc__k_x_column = pd.read_table(
            hcc__k_x_column_file_path,
            index_col=0,
        )

        gps_map.set_element_states(
            wt_or_h,
            hcc__k_x_column.loc['K{}'.format(hcc_k)],
            bandwidth_factor=bandwidth_factor,
        )

        gps_map.plot_gps_map(
            wt_or_h,
            title=setting.NAME,
            element_marker_size=element_marker_size,
            html_file_path='{}/map_{}.html'.format(
                path_dict['{}_map_directory_path'.format(wt_or_h)],
                hcc_k,
            ),
            plotly_file_path=path_dict['{}_state_map_plotly_file_path'.format(wt_or_h)],
        )

In [ ]:
ccal.dump_gps_map(
    gps_map,
    path_dict['gps_map_file_path'],
)